In [1]:
import xml.etree.ElementTree as ET
from lxml import etree
import xmlschema

#diagram validation functions for the two DAOMod diagram types
def validate_against_schema(file_name,schema_name, diagram_name):
    diagram_xml = etree.parse(file_name)
    validation = xmlschema.XMLSchema(schema_name)    
    if validation.is_valid(diagram_xml):
        print(f'The XML file structure complies with the {diagram_name} schema')
        return True
    else:
        print(f'The XML file is not a valid {diagram_name}')
        return False

def split_and_add_to_list(strings):
    result = []
    for string in strings:
        substrings = string.split()
        result.extend(substrings)
    result_set = set(result)
    return result_set

def compare_subsets(diagram,name1, query1,name2, query2):
    id_list1 = diagram.xpath(query1)
    id_list2 = diagram.xpath(query2)
    if split_and_add_to_list(id_list1).issubset(split_and_add_to_list(id_list2)):
        print(f"the set of {name1} \n {split_and_add_to_list(id_list1)} is a subset of {name2}:\n {split_and_add_to_list(id_list2)}\n")
        return True
    else:
        print(f"invalid relation:{split_and_add_to_list(id_list1)} is not a subset of {split_and_add_to_list(id_list2)}\n")
        return False

def check_root(diagram):
    root_elements=[]
    for element in split_and_add_to_list(diagram.xpath('//*[refinement_level="0"]/@id')):
        root_elements.append(element)
    if len(root_elements) == 1:
        print(f"root goal {root_elements[0]} is unique")
        if len(diagram.xpath('//*[refinement_level="0"]/@decomposed_from'))==0:
            print("the set of elements decomposed from the root is empty.")
            return True
    else: 
        print("\n the diagram is not valid.")
        print("the number of elemenets decomposed from the root element are: ",len(diagram.xpath('//*[refinement_level="0"]/@decomposed_from')))
        print("the elements with refinement_level=0 are: ",root_elements)
        return False
        
#functions to check federation and aggregation relations in DO Diagrams
def check_relation_graphs(diagram, rel_attribute, rel_name):
    for role_elem in diagram.xpath("//Role | //Committee"):
        role_id = role_elem.get("id")
        level = role_elem.get(rel_attribute)
        
        if level is None:
            print(f"Role/Committee with id {role_id} does not have a level for {rel_name}")
            continue

        level = int(level)
        
        relation = role_elem.xpath(f"{rel_name}/text()")
        if relation:
            for relator_id in relation:
                relator_elems = diagram.xpath(f'//*[@id="{relator_id}"]')
                if not relator_elems:
                    print(f"No element found with id {relator_id}")
                    continue
                
                relator_elem = relator_elems[0]
                relator_level = relator_elem.get(rel_attribute)
                
                if relator_level is None:
                    print(f"Relator with id {relator_id} does not have an {rel_attribute}")
                    continue
                
                relator_level = int(relator_level)
                
                print(f"Checking Role/Committee with id {role_id}")
                print(f"Role/Committee {rel_name} level: {level}")
                print(f"Relator id: {relator_id}, Relator level: {relator_level}")

                if level < relator_level:
                    print(f"{rel_name} relation is valid\n")
                    return True
                else:
                    print(f"{rel_name} relation violation\n")
                    return False

In [2]:
#functions to check DAOMod diagrams
def check_req_diagram(diagram_file, schema_file, diagram_name):
    conditions =[]
    diagram = etree.parse(diagram_file)
    conditions.append(validate_against_schema(diagram_file,schema_file,diagram_name))
    #print(validate_against_schema(diagram_file,schema_file,diagram_name))
    conditions.append(check_root(diagram))
    #OrganizationGoal decomposition validation
    conditions.append(compare_subsets(diagram,"IDs Organization Goals are decomposed from ", '//organizationGoal/@decomposed_from',"Goal IDs", '//goal/@id') )
    #functional area decomposition validation
    conditions.append(compare_subsets(diagram,"IDs functional areas are decomposed from ", '//decisionMakingMethod/@decomposed_from|//tokenManager/@decomposed_from| //treasury/@decomposed_from',"Organization Goal IDs", '//organizationGoal/@id'))
    #token decomposition validation
    conditions.append(compare_subsets(diagram,"IDs tokens are decomposed from ", '//token/@decomposed_from', "treasury or token manager IDS",'//tokenManager/@id|//treasury/@id'))
    #mechanism decomposition validation
    conditions.append(compare_subsets(diagram,"IDs mechanisms are decomposed from ", '//mechanism/@decomposed_from', "functional area and token IDs",'//tokenManager/@id | //treasury/@id | //token/@id | //decisionMakingMethod/@id'))
    #transaction decomposition validation
    conditions.append(compare_subsets(diagram,"IDs transactions are decomposed from", '//transaction/@decomposed_from', " asset IDs", '//asset/@id'))
    #asset decompostion validation
    conditions.append(compare_subsets(diagram,"IDs asset elements are decomposed_from", '//asset/@decomposed_from'," Onchain element IDs", '//*[@execution="onchain"]/@id|//token/@id|treasury/@id|tokenManager/@id'))
    #proposal association validation
    conditions.append(compare_subsets(diagram,"IDs Proposal elements are associated_to", '//proposal/@associated_to'," DecisionMakingMethod IDs", '//decisionMakingMethod/@id'))
    #resource association validation
    conditions.append(compare_subsets(diagram,"IDs Resource elements are associated_to", '//resource/@associated_to'," Token IDs", '//token/@id'))
    #quality goal, emotional goal validation
    conditions.append(compare_subsets(diagram,"IDs Quality Goals and Emotional Goals are associated_to", '//qualitygoal/@associated_to|//emotionalgoal/@associated_to'," Decomposable element IDs", '//tokenManager/@id | //treasury/@id | //token/@id | //decisionMakingMethod/@id | //organizationGoal/@id | //goal/@id | //dataStore/@id | //mechanism/@id | //asset/@id| //transaction/@id'))
    # agent association validation
    conditions.append(compare_subsets(diagram,"IDs Agents are associated_to", '//agent/@associated_to'," Decomposable element IDs or Emotional Goals", '//tokenManager/@id | //treasury/@id | //token/@id | //decisionMakingMethod/@id | //organizationGoal/@id | //goal/@id | //dataStore/@id | //mechanism/@id | //asset/@id| //transaction/@id | //emotionalgoal/@id'))
    # goal decomposition validation
    conditions.append(compare_subsets(diagram,"IDs Goals are decomposed from", '//goal/@decomposed_from'," Decomposable element IDs", '//tokenManager/@id | //treasury/@id | //token/@id | //decisionMakingMethod/@id | //organizationGoal/@id | //goal/@id | //dataStore/@id | //mechanism/@id | //asset/@id| //transaction/@id'))
    #checking that all conditions are met
    for condition in conditions:
        if condition != True:
            print(f"error at condition {conditions.index(condition)}")
            print(condition)
            return False
    print(f"All conditions are valid for the diagram {diagram_file}")
    return True
    
def check_dec_org_diagram(diagram_file, schema_file, diagram_name):
    conditions =[]

    diagram = etree.parse(diagram_file)
    conditions.append(validate_against_schema(diagram_file,schema_file,diagram_name))
    conditions.append(compare_subsets(diagram,"IDs that roles or committees are associated to", '//Role/associated_to/text()| //Committee/associated_to/text()',"Permission or Governance Area IDs", '//GovernanceArea/@gov_area_ID | //Permission/@permission_ID') )   
    conditions.append(compare_subsets(diagram,"IDs of elements that roles aggregate into", '//Role/aggregates/text()',"Roles' IDs", '//Role/@id'))    
    conditions.append(compare_subsets(diagram,"IDs of elements that committees aggregate into", '//Committee/aggregates/text()',"Committee' IDs", '//Committee/@id')    )
    conditions.append(compare_subsets(diagram,"IDs of control relations", '//Role/is_controlled_by/text()|//Committee/is_controlled_by/text()',"Role or Committee IDs", '//Role/@id|//Committee/@id'))  
    conditions.append(check_relation_graphs(diagram,"aggregation_level", "aggregates"))
    conditions.append(check_relation_graphs(diagram,"federation_level", "member_of"))
    for condition in conditions:
        if condition != True:
            print(f"error at condition {conditions.index(condition)}")
            print(condition)
            return False
    print(f"All conditions are valid for the diagram {diagram_file}")
    return True

In [3]:

#Checking Circles UBI DAO diagram
print("\n----Checking Circles UBI DAO diagram----\n")
assert check_req_diagram('Circles_DAO_RD.xml','DAOMod_XSD_Extended Requirement Diagram.xsd',"Requirement Diagram")
print("\n----Checking CommonsHood DAO platform diagram----\n")
assert check_req_diagram('CH_DAO_RD.xml','DAOMod_XSD_Extended Requirement Diagram.xsd',"Requirement Diagram")
print("\n----Checking White Rabbit DAO diagram----\n")
assert check_req_diagram('WR_DAO_RD.xml','DAOMod_XSD_Extended Requirement Diagram.xsd',"Requirement Diagram")



----Checking Circles UBI DAO diagram----

The XML file structure complies with the Requirement Diagram schema
root goal og1 is unique
the set of elements decomposed from the root is empty.
the set of IDs Organization Goals are decomposed from  
 set() is a subset of Goal IDs:
 {'g1'}

the set of IDs functional areas are decomposed from  
 {'og1'} is a subset of Organization Goal IDs:
 {'og1'}

the set of IDs tokens are decomposed from  
 {'tm1', 't1'} is a subset of treasury or token manager IDS:
 {'tm1', 't1'}

the set of IDs mechanisms are decomposed from  
 {'EUROE', 'GC', 't1', 'CT', 'PS'} is a subset of functional area and token IDs:
 {'EUROE', 'GC', 't1', 'CT', 'tm1', 'PS', 'dmm2', 'dmm1'}

the set of IDs transactions are decomposed from 
 {'a2', 'a1'} is a subset of  asset IDs:
 {'a2', 'a1'}

the set of IDs asset elements are decomposed_from 
 {'g1'} is a subset of  Onchain element IDs:
 {'EUROE', 'm6', 'g1', 'GC', 't1', 'm3', 'CT', 'm4', 'tm1', 'PS', 'm2', 'dmm1', 'm1', 'm5', 

In [4]:
print("\n----Checking Circles UBI DAO diagram----\n")
assert check_dec_org_diagram('Circles_DAO_DO.xml', 'DAOMod_XSD_Decentralized_Organization.xsd', "Decentralized Organization Diagram")
print("\n----Checking CommonsHood DAO platform diagram----\n")
assert check_dec_org_diagram('WR_DAO_DO.xml', 'DAOMod_XSD_Decentralized_Organization.xsd', "Decentralized Organization Diagram")
print("\n----Checking White Rabbit DAO diagram----\n")
assert check_dec_org_diagram('CH_DAO_DO.xml', 'DAOMod_XSD_Decentralized_Organization.xsd', "Decentralized Organization Diagram")


----Checking Circles UBI DAO diagram----

The XML file structure complies with the Decentralized Organization Diagram schema
the set of IDs that roles or committees are associated to 
 {'set_limits_to_group_currency_minting', 'include_exclude_group_members', 'GroupMembership', 'TreasuryManagement', 'evaluate_service_provision', 'set_contribution_attestation', 'Attestations', 'DAOUpgrades', 'upgrade_DAO_smart_contracts', 'ServiceProvision', 'EmergencyResponse', 'propose_service_provision', 'add_remove_allowed_collateral_type', 'suspension_of_the_group_currency', 'member_data_management', 'set_membership_requirements', 'update_user_profile', 'funding_request_submission', 'activate_suspend_service_provision', 'upgrade_Group_Currency_smart_contracts', 'funding_request_assessment'} is a subset of Permission or Governance Area IDs:
 {'set_limits_to_group_currency_minting', 'include_exclude_group_members', 'GroupMembership', 'TreasuryManagement', 'evaluate_service_provision', 'set_contributi